In [1]:
# 科学运算
import pandas as pd 
import numpy as np 
# 绘图
import matplotlib.pyplot as plt 
import seaborn as sns 
# 无视警告
import warnings 
warnings.filterwarnings('ignore')
# %matplotlib inline 

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei'] #设置字体
# sns.set(font='SimHei') #设置中文编码
plt.rcParams['font.family'] = 'SimHei' #选择中文字体

In [2]:
# 读取数据
train = pd.read_csv('B题-示例数据/示例数据/order_train0.csv')
predict = pd.read_csv('B题-示例数据/示例数据/predict_sku0.csv')

In [3]:
# 数据统计
def summary_stats_table(data):
    '''
    a function to summerize all types of data
    分类型按列的数据分布与异常值统计
    '''
    # count of nulls
    # 空值数量
    missing_counts = pd.DataFrame(data.isnull().sum())
    missing_counts.columns = ['count_null']

    # numeric column stats
    # 数值列数据分布统计
    num_stats = data.select_dtypes(include=['int64','float64']).describe().loc[['count','min','max','25%','50%','75%']].transpose()
    num_stats['dtype'] = data.select_dtypes(include=['int64','float64']).dtypes.tolist()

    # non-numeric value stats
    # 非数值列数据分布统计
    non_num_stats = data.select_dtypes(exclude=['int64','float64']).describe().transpose()
    non_num_stats['dtype'] = data.select_dtypes(exclude=['int64','float64']).dtypes.tolist()
    non_num_stats = non_num_stats.rename(columns={"first": "min", "last": "max"})

    # merge all 
    # 聚合结果
    stats_merge = pd.concat([num_stats, non_num_stats], axis=0, join='outer', ignore_index=False, keys=None,
              levels=None, names=None, verify_integrity=False, copy=True, sort=False).fillna("").sort_values('dtype')

    column_order = ['dtype', 'count', 'count_null','unique','min','max','25%','50%','75%','top','freq']
    summary_stats = pd.merge(stats_merge, missing_counts, left_index=True, right_index=True, sort=False)[column_order]
    return(summary_stats)

In [4]:
summary_stats_table(train)

,dtype,count,count_null,unique,min,max,25%,50%,75%,top,freq
sales_region_code,int64,59769.0,0,,101.0,105.0,102.0,103.0,105.0,,
item_code,int64,59769.0,0,,20001.0,22084.0,20512.0,20973.0,21518.0,,
first_cate_code,int64,59769.0,0,,301.0,308.0,303.0,306.0,306.0,,
second_cate_code,int64,59769.0,0,,401.0,412.0,404.0,407.0,408.0,,
ord_qty,int64,59769.0,0,,1.0,12480.0,10.0,29.0,101.0,,
item_price,float64,59769.0,0,,1.0,47911.0,597.0,880.28,1291.0,,
order_date,object,59769.0,0,1168,,,,,,2017/5/25,168
sales_chan_name,object,59769.0,0,2,,,,,,offline,44084


# 1.数据预处理

## 1.1 重复值检测及处理

In [5]:
# 查找重复值
train.duplicated().sum()

3

In [6]:
# 创建包含重复数据行的新DataFrame
duplicate_rows = train[train.duplicated(keep=False)]
duplicate_rows

,order_date,sales_region_code,item_code,first_cate_code,second_cate_code,sales_chan_name,item_price,ord_qty
33927,2017/11/3,102,20135,303,401,offline,859.0,3
33937,2017/11/3,102,20135,303,401,offline,859.0,3
52393,2018/9/28,103,20531,307,403,offline,1239.0,18
52394,2018/9/28,103,20531,307,403,offline,1239.0,18
56248,2018/11/9,103,20661,302,408,offline,926.0,10
56249,2018/11/9,103,20661,302,408,offline,926.0,10


In [7]:
# 删除重复值
train = train.drop_duplicates()

## 1.2 缺失值检测及处理

In [8]:
# 检测缺失值
train.isna().sum()

order_date           0
sales_region_code    0
item_code            0
first_cate_code      0
second_cate_code     0
sales_chan_name      0
item_price           0
ord_qty              0
dtype: int64

In [9]:
# 日期预处理
train['order_date'] = pd.to_datetime(train['order_date'])

## 1.3 异常值检测及处理


在数据整体分布分析中
**一级编码会对应不同的二级编码，每个一级编码所包含的二级编码都不同**

**一级编码**（first_cate_code）共：301 - 308  

**二级编码**（second_cate_code）共：401 - 412  

在初步观察发现：**306的占比数量特别多，407的占比数量特别多**，在下面分析可得出：代表着306-407这一细类产品特别多

​	如（——代表对应的意思）：

​	301——405

​	302——408

​	303——401，406，410，411（401的数量比其他三类加起来还多）

​	304——409

​	305——412

​	306——402，407（407的数量特别多）

​	307——403

​	308——404

这些都可以说，观察数据中是否有对应一、二级编码错误或一个产品对应不同一二级编码

价格、需求量等异常值应该怎么检测，如何处理

In [10]:
# 保存数据
train.to_csv('train_cleaned.csv',index=False)